In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("Financial"))))

In [2]:
from Collection.Financial import FMP as fp

# Collect

In [3]:
fp.information()


        함수에 대한 설명은 아래와 같습니다. 

        라이브러리 내 주요 클래스는 fmp_extact입니다. 

        get_jsonparsed_data()는 데이터를 파싱하는 함수입니다. 

        extractor()은 데이터를 json형태로 가지고 오는 함수입니다. 

        url_generator()은 FMP 사이트에 접속하여 데이터를 분리하는 함수입니다. 

        ending_period_extact()는 날짜를 표준화하는 함수입니다. 

        report_type_extract()는 들어오는 값에 따라 연간인지 분기인지 구분하는 함수입니다. 

        GetExcel()은 추출한 데이터를 저장하는 함수입니다. 

        cleanse()는 데이터를 가공하는 함수입니다. 

        get_symbols()는 데이터를 사이트로부터 가져오는 함수입니다. 

        make_clean()은 위의 함수들을 순차적으로 실행하여 데이터를 추출 후 저장하는 함수입니다.
        


In [4]:
fps = fp.fmp_extract()

In [5]:
country_lst = ["호주", "스위스"]

In [6]:
for idx in country_lst:
    fps.get_symbols(idx)

100%|██████████| 1000/1000 [2:16:08<00:00,  8.17s/it] 
0it [00:00, ?it/s]


KeyError: 'symbol'

# Transform

In [4]:
fps = fp.fmp_extract()

In [5]:
filepath = os.listdir("og/fmp/")

In [6]:
for idx in filepath:
    fps.cleanse(f"og/fmp/{idx}", f"finished/{idx}")

finished/Original_미국_2000.xlsx이 완료되었습니다.
finished/Original_미국_4000.xlsx이 완료되었습니다.
finished/Original_미국_6000.xlsx이 완료되었습니다.
finished/Original_스위스_834.xlsx이 완료되었습니다.
finished/Original_영국_2000.xlsx이 완료되었습니다.
finished/Original_영국_2624.xlsx이 완료되었습니다.
finished/Original_인도_2000.xlsx이 완료되었습니다.
finished/Original_인도_2090.xlsx이 완료되었습니다.
finished/Original_캐나다_1846.xlsx이 완료되었습니다.


# 3rd DATA EXTRACT

In [18]:
import pandas as pd
usa_df = pd.read_excel("3rd_data/tb_hb_idn_plcfi_d.xlsx")

In [19]:
usa_symbol_lst = list(set(usa_df["lstng_cd"]))

In [20]:
from tqdm import tqdm
df = pd.DataFrame()

In [21]:
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [22]:
for symbol in tqdm(usa_symbol_lst):
    try:
        url = (f"https://financialmodelingprep.com/api/v4/company-core-information?symbol={symbol}&apikey=89d4891348727c3950b79b9067127c3f")
        symbol_dict = get_jsonparsed_data(url)[0]
        df = df.append(symbol_dict, ignore_index=True)
    except: pass

  0%|          | 0/803 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_3184\3661239991.py:23: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())
  1%|          | 5/803 [00:03<10:42,  1.24it/s]C:\Users\user\AppData\Local\Temp\ipykernel_3184\4193271034.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(symbol_dict, ignore_index=True)
  1%|          | 6/803 [00:04<10:40,  1.24it/s]C:\Users\user\AppData\Local\Temp\ipykernel_3184\3661239991.py:23: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())
C:\Users\user\AppData\Local\Temp\ipykernel_3184\4193271034.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [24]:
df

,cik,symbol,exchange,sicCode,sicGroup,sicDescription,stateLocation,stateOfIncorporation,fiscalYearEnd,businessAddress,mailingAddress,taxIdentificationNumber,registrantName
0,0001396440,MAIN,NYSE,,Asset Management,,TX,MD,12-31,"1300 POST OAK BOULEVARD,HOUSTON TX 77056,(713)...","1300 POST OAK BOULEVARD,HOUSTON TX 77056",41-2230745,Main Street Capital Corporation
1,0001709505,BEST,NYSE,4210,Trucking,TRUCKING & COURIER SERVICES (NO AIR),None,E9,12-31,"2/F, BLOCK A, HUAXING MODERN INDUSTRY,HANGZHOU...","2/F, BLOCK A, HUAXING MODERN INDUSTRY,HANGZHOU...",00-0000000,BEST Inc.
2,0001840502,TBLA,NASDAQ,7370,Internet Content & Information,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",NY,L3,12-31,"16 MADISON SQUARE WEST, 7TH FL.,NEW YORK NY 10...","16 MADISON SQUARE WEST, 7TH FL.,NEW YORK NY 10010",00-0000000,Taboola.com Ltd.
3,0001128361,HOPE,NASDAQ,6021,Banks—Regional,NATIONAL COMMERCIAL BANKS,CA,DE,12-31,"3200 WILSHIRE BLVD,LOS ANGELES CA 90010,213639...","3200 WILSHIRE BLVD,LOS ANGELES CA 90010",95-4849715,"Hope Bancorp, Inc."
4,0001597553,SAGE,NASDAQ,2834,Biotechnology,PHARMACEUTICAL PREPARATIONS,MA,DE,12-31,"215 FIRST STREET,CAMBRIDGE MA 02142,617-299-8380","215 FIRST STREET,CAMBRIDGE MA 02142",00-0000000,"Sage Therapeutics, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0000720858,ITIC,NASDAQ,6361,Insurance—Specialty,TITLE INSURANCE,NC,NC,12-31,"121 N COLUMBIA ST,CHAPEL HILL NC 27514,9199682200","121 NORTH COLUMBIA STREET,CHAPEL HILL NC 27514",56-1110199,Investors Title Company
163,None,None,AMEX,None,None,None,None,None,None,", ,",",",-,Pacer Benchmark Industrial Real Estate SCTR ETF
164,None,None,NASDAQ,None,Shell Companies,None,CA,None,None,", ,",",",-,Dragoneer Growth Opportunities Corp. II
165,0001830043,BMBL,NASDAQ,7370,Software—Application,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",TX,,12-31,"1105 WEST 41ST STREET,AUSTIN TX 78756,512-696-...","1105 WEST 41ST STREET,AUSTIN TX 78756",85-3604367,Bumble Inc.


In [16]:
df.to_excel("미국_3차년도_재무정보_일반.xlsx", index=False)